### Import Libraries

In [1]:
import numpy as  np
import matplotlib.pyplot as plt
import cv2
import sklearn as sk
import glob
import random as rd
from skimage.feature import hog
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import time
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from scipy.ndimage.measurements import label
import moviepy
from moviepy.editor import VideoFileClip
import pickle

In [2]:
# functions for extracting features

# to get bin spatial features
def bin_spatial(img,size=(32, 32)):
    
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img,size).ravel()
    # Return the feature vector
    return features

#to get color histogram features
def color_hist(img, nbins=32, bins_range=(0, 256),visualize=True):
    # Compute the histogram of the Passed channels separately
    
    
    channel1_hist = np.histogram(img[:,:,0],bins=nbins,range=bins_range)
    channel2_hist = np.histogram(img[:,:,1],bins=nbins,range=bins_range)
    channel3_hist = np.histogram(img[:,:,2],bins=nbins,range=bins_range)
    
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    
    if visualize == False :
        return hist_features
    
    else :
        
        # Generating bin centers
        edges = channel1_hist[1]
        bin_centers = (edges[1:] + edges[0:len(edges)-1])/2
               
        # Return the individual histograms, bin_centers and feature vector
        return hist_features,channel1_hist, channel2_hist, channel3_hist, bin_centers, hist_features

#to get HOG features
def get_hog_features(img, orient, pix_per_cell, cell_per_block,visualize=False, feature_vector=True):
    
    if visualize == False:
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), block_norm= 'L2-Hys',transform_sqrt=True, 
                       visualise=visualize, feature_vector=feature_vector)
        return features

    # Otherwise call with one output
    else:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), block_norm= 'L2-Hys',
                                  transform_sqrt=True, 
                                  visualise=visualize, feature_vector=feature_vector)
        return features, hog_image

#function to combine all the features
def combine_features(img, color_space='BGR', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        bin_spatialFlag=True, color_histFlag=True,visualize=False):
    
        
    # apply color conversion if other than 'BGR'
    if color_space != 'BGR':
        color_change= eval("cv2.COLOR_BGR2" + color_space)
        changeImage = cv2.cvtColor(img,color_change)
          
    else:
        changeImage = np.copy(img)      

    if bin_spatialFlag == True:
            
        spatial_features = bin_spatial(changeImage, size=spatial_size)
            
    if color_histFlag == True:
            
        hist_features = color_hist(changeImage, nbins=hist_bins,visualize=visualize)
            
    # run get_hog_features to get HOG features
    if hog_channel == 'ALL':
        hog_features = []
        for channel in range(changeImage.shape[2]):
            hog_features.append(get_hog_features(changeImage[:,:,channel], 
                                        orient, pix_per_cell, cell_per_block, 
                                        visualize=False, feature_vector=True))
        hog_feature = np.ravel(hog_features)        
            
    else:
        hog_feature = get_hog_features(changeImage[:,:,hog_channel], orient, 
                            pix_per_cell, cell_per_block, visualize=False, feature_vector=True)
        
    if bin_spatialFlag == True and color_histFlag==True :
        feature=np.concatenate((hog_feature,spatial_features, hist_features))
        
    elif bin_spatialFlag==True :
        feature=np.concatenate((hog_feature,spatial_features))
            
    elif color_histFlag==True:
        feature=np.concatenate((hog_feature,hist_features))
            
    else:
        feature=hog_feature
    
    # Return list of feature vectors
    return feature
    

In [3]:
color_space = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
spatial_size = (32, 32) # Spatial binning dimensions
hist_bins = 32    # Number of histogram bins
orient = 9  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"
spatial_feature = True # Spatial features on or off
hist_feature = True # Histogram features on or off


In [4]:
# functions for sliding windows
# Define a function that takes an image,
# start and stop positions in both x and y, 
# window size (x and y dimensions),  
# and overlap fraction (for both x and y)
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step) 
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step) 
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list

# Define a function to draw bounding boxes
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy

In [5]:
def search_cars(img, windows, clf, scaler, color_space='BGR', 
                    spatial_size=(32, 32), hist_bins=32, 
                    hist_range=(0, 256), orient=9, 
                    pix_per_cell=8, cell_per_block=2, 
                    hog_channel=0, spatial_feat=True, 
                    hist_feat=True, visualise=True):

    #1) Create an empty list to receive positive detection windows
    on_windows = []
    #2) Iterate over all windows in the list
    for window in windows:
        #3) Extract the test window from original image
        resize_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))      
        #4) Extract features for that window using combine_features()
        features = combine_features(resize_img, color_space=color_space, spatial_size=spatial_size,
                        hist_bins=hist_bins, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, hog_channel=hog_channel,
                        bin_spatialFlag=spatial_feat, color_histFlag=hist_feat)
        
        #5) Scale extracted features to be fed to classifier
        scaled_features = scaler.transform(np.array(features).reshape(1, -1))
        #6) Predict using your classifier
        prediction = clf.predict(scaled_features)
        #7) If positive (prediction == 1) then save the window
        if prediction == 1:
            on_windows.append(window)
    #8) Return windows for positive detections
    return on_windows


In [6]:

#
def get_heatMap (heatmap, boundingBoxes):
    # iterate through the bounding boxes
    for box in boundingBoxes:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1
    # return the updated heat map
    return heatmap

def define_threshold_heatMap(heatmap,threshold=2):
    #set zero , were values less than threshold
    heatmap[heatmap<=threshold]=0
    # return updated heatmap
    return heatmap

def apply_heatMap(img,boundingBoxes,threshold,visualise=False):
    # get an array similar to image with zero's , use only single channel
    heatmap = np.zeros_like (img[:,:,0],np.float)
    
    #get heat map for the bounding boxes
    heatmap_ret = get_heatMap(heatmap,boundingBoxes)
    
    #apply threshold to heatmap returned from above function
    heatmap_thresh = define_threshold_heatMap(heatmap_ret,threshold)
    
    #apply numpy.clip so that all values are with in range of 0-255
    heatmap_res= np.clip(heatmap_thresh, 0, 255)
    
    # Find final boxes from heatmap using label function
    labels = label(heatmap_res)
        
    result= draw_labeled_bboxes(img, labels)
    
    if visualise == True :
        
        return result,heatmap_res
    
    else:
        
        return result

def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
    # Return the image
    return img    
    

In [7]:

def pipeline(img):
    global load_model
    global classifier
    global data_scalar
    #convert image to BGR 
    if load_model== True :
        model_data= pickle.load(open("model_saver.p","rb"))
        classifier = model_data["classifier"]
        data_scalar = model_data["data_scaler"]
        load_model =False
    
    imgBGR = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
    draw_image = np.copy(imgBGR)
    y_start_stop1= [400,500]
    y_start_stop2= [450,500]
    y_start_stop3= [450,600]
    x_start_stop= [(img.shape[1]//2), img.shape[1]]
    threshold=3
    
#     slide1 = slide_window(draw_image, x_start_stop=x_start_stop, y_start_stop=y_start_stop, 
#                     xy_window=(64,64), xy_overlap=(0.15, 0.15))
#     slide2 = slide_window(draw_image, x_start_stop=x_start_stop, y_start_stop=y_start_stop, 
#                     xy_window=(96,96), xy_overlap=(0.2, 0.2))
#     slide3 = slide_window(draw_image, x_start_stop=x_start_stop, y_start_stop=y_start_stop, 
#                     xy_window=(128,128), xy_overlap=(0.25, 0.25))
#     slide4 = slide_window(draw_image, x_start_stop=x_start_stop, y_start_stop=y_start_stop, 
#                     xy_window=(32,32), xy_overlap=(0.5, 0.5))

    slide1 = slide_window(draw_image, x_start_stop=x_start_stop, y_start_stop=y_start_stop1, 
                    xy_window=(64,64), xy_overlap=(0.80, 0.80))
    slide2 = slide_window(draw_image, x_start_stop=x_start_stop, y_start_stop=y_start_stop2, 
                    xy_window=(96,96), xy_overlap=(0.80, 0.80))
    slide3 = slide_window(draw_image, x_start_stop=x_start_stop, y_start_stop=y_start_stop3, 
                    xy_window=(32,32), xy_overlap=(0.80, 0.80))
    slide4 = slide_window(draw_image, x_start_stop=x_start_stop, y_start_stop=y_start_stop3, 
                    xy_window=(128,128), xy_overlap=(0.80, 0.80))

    
    windows = slide1 + slide2 #+  slide3 

#     print (" windows length ", len(windows))
#     print (" X start " , x_start_stop)
    
    hot_windows = search_cars(draw_image, windows, classifier, data_scalar, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feature, 
                        hist_feat=hist_feature)                       

    finalImage = apply_heatMap (draw_image,hot_windows,threshold,visualise=False)
    #convert back to RGB
    result = cv2.cvtColor(finalImage,cv2.COLOR_BGR2RGB)
    
    return result

In [ ]:
# test pipeline
#  = plt.imread('test_images/test1.jpg')
load_model =True
loc = "Testing_Images/"
# loc = "test_images/test"
# traverese from the previous list
for i in range(935,958):
    fname=loc + str(i) + ".jpg"
    img = plt.imread(fname)
    res = pipeline(img)
    
    res_fig , res_axis = plt.subplots(1,2, figsize=(16, 8))
       
    res_axis[0].imshow(img)
    res_axis[0].set_title("Original " + fname.split('/')[-1] , fontsize=10)
    res_axis[1].imshow(res)
    res_axis[1].set_title("With Cars " + fname.split('/')[-1] , fontsize=10)


In [8]:

def save_clip(img):
    global i
    plt.imsave("Testing_Images/" + str(i) + ".jpg",img)
    i += 1
    return img

i =0 
load_model =True
# create video using pipeline
output = 'lanelinesvideo_output.mp4'
clip2 = VideoFileClip("video_output_lanes.mp4")
# white_clip = clip1.fl_image(save_clip) #NOTE: this function expects color images!!
white_clip = clip2.fl_image(pipeline) #NOTE: this function expects color images!!
%time white_clip.write_videofile(output, audio=False)

[MoviePy] >>>> Building video lanelinesvideo_output.mp4
[MoviePy] Writing video lanelinesvideo_output.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [29:46<00:01,  1.34s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: lanelinesvideo_output.mp4 

Wall time: 29min 48s
